In [1]:
%pwd

'/Users/yangbo/Documents/projects/qamp_gse/test/ipynb/gse_vqe/backends/ibmq_jakarta'

# COMMON --- LIBRARIES AND UTILS

In [2]:
import sys, pickle, time, importlib
sys.path.append("../")
from pprint import pprint
import numpy as np
np.random.seed(42)
np.set_printoptions(threshold=sys.maxsize)
import scipy as spy
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [3]:
path_utils = "../"
sys.path.append(path_utils)
from utils import *
from dsp import *
from gse import *

# Load the settings for real device implementation

In [4]:
import pickle
with open("pkls/2_make_qrem_filter.pkl", "rb") as f:
    records = pickle.load(f)
    meas_fitter = records["meas_fitter"]

In [5]:
import pickle
with open("pkls/1_run_gse_r1-r2-r3.pkl", "rb") as f:
    records = pickle.load(f)

In [6]:
import pickle
with open("pkls/0_settings.pkl", "rb") as f:
    settings = pickle.load(f)
backend = settings["backend"]
H = settings["H"]

In [7]:
sigma_list = records["sigma_list"]
sigma_list_inv = records["sigma_list_inv"]
pauli_list = records["pauli_list"]
job_id_qcs_H_tilde = records["job_id_qcs_H_tilde"]
job_id_qcs_S_tilde = records["job_id_qcs_S_tilde"]
# job_qcs_H_tilde = backend.retrieve_job(job_id_qcs_H_tilde) ### if the results are stored on the cloud ###
# job_qcs_S_tilde = backend.retrieve_job(job_id_qcs_S_tilde) ### if the results are stored on the cloud ###
print(job_id_qcs_H_tilde)
print(job_id_qcs_S_tilde)

6408869c0ad55fd6d8f0fb18
640886a71e92645b7ee6b622


### Retrieve jobs and make histograms

In [10]:
# results_H_tilde = job_qcs_H_tilde.result() ### if the results are stored on the cloud ###
# results_S_tilde = job_qcs_S_tilde.result() ### if the results are stored on the cloud ###
hists_H_tilde = results_H_tilde.get_counts()
hists_S_tilde = results_S_tilde.get_counts()

In [8]:
import json
from qiskit.result import Result

f = open("pkls/job-"+job_id_qcs_H_tilde+"/"+job_id_qcs_H_tilde+"-output.json")
data = json.load(f)
results_H_tilde = Result.from_dict(data)

f = open("pkls/job-"+job_id_qcs_S_tilde+"/"+job_id_qcs_S_tilde+"-output.json")
data = json.load(f)
results_S_tilde = Result.from_dict(data)

### Apply QREM (if required)

In [12]:
results_H_tilde_qrem = meas_fitter.filter.apply(connect_cregs(results_H_tilde))
results_S_tilde_qrem = meas_fitter.filter.apply(connect_cregs(results_S_tilde))
hists_H_tilde_qrem = results_H_tilde_qrem.get_counts()
hists_S_tilde_qrem = results_S_tilde_qrem.get_counts()

### Compute the mitigated expectation value
Here we output raw eigenvalues and eigenvectors of the generalize eigenvalue problem in GSE:
$$\mathcal{H} \vec{\alpha}=E \mathcal{S} \vec{\alpha}$$
After this, we remove the unphysical eigenvalues and choose the minimum one among the remainings.

In [13]:
eig_vals, eig_vecs, H_tilde, S_tilde = compute_energy_gse_dsp_ancilla(hists_H_tilde=hists_H_tilde_qrem,
                                                                      hists_S_tilde=hists_S_tilde_qrem,
                                                                      hamiltonian=H,
                                                                      sigma_list=sigma_list,
                                                                      pauli_list=pauli_list,
                                                                      hist_type="raw",
                                                                      return_all=True)
max_energy = len(H)
eig_vals_lower_bounded = eig_vals.real[- max_energy < eig_vals.real]
eig_vals_bounded = eig_vals_lower_bounded[eig_vals_lower_bounded < max_energy]
energy = np.sort(eig_vals_bounded)[0]
print("Estimated ground state energy by GSE (fault subspace):", energy)
print()
print("physically meaningful eigen values")
pprint(eig_vals_bounded)
# print()

Estimated ground state energy by GSE (fault subspace): -2.3074444180792013

physically meaningful eigen values
array([-2.30744442, -0.5253814 ])


# Save Data

In [14]:
records = dict()
records["qrem_r1-r2-r3"] = {"eig_vals": eig_vals,
                            "eig_vecs": eig_vecs,
                            "H_tilde": H_tilde,
                            "S_tilde": S_tilde,
                            "energy": energy,}

In [15]:
import pickle
with open("pkls/2_apply_qrem_gse_r1-r2-r3.pkl", "wb") as f:
    pickle.dump(records, f)

# Qiskit versions

In [16]:
import qiskit.tools.jupyter
%qiskit_version_table